<a href="https://colab.research.google.com/github/cetinus-gulin/ie423/blob/main/task5-6-7/task7-gulin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialize

In [1]:
import pandas as pd

# Load data

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

#dfPurchase = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
dfPurchase = pd.read_csv('train.csv')
dfPurchase.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422
3,1000001,P00085442,F,0-17,10,A,2,0,12,14.0,NaN,1057
4,1000002,P00285442,M,55+,16,C,4+,0,8,NaN,NaN,7969


# Prepare data

In [3]:
# Drop missing values
dfPurchase.dropna(inplace=True)
dfPurchase.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
6,1000004,P00184942,M,46-50,7,B,2,1,1,8.0,17.0,19215
13,1000005,P00145042,M,26-35,20,A,1,1,1,2.0,5.0,15665
14,1000006,P00231342,F,51-55,9,A,1,0,5,8.0,14.0,5378
16,1000006,P0096642,F,51-55,9,A,1,0,2,3.0,4.0,13055


In [4]:
# Convert to list format
dfPurchaseList=dfPurchase.groupby('User_ID').Product_ID.apply(list)
dfPurchaseList

User_ID
1000001    [P00248942, P00085942, P00102642, P00110842, P...
1000002    [P00289342, P00034742, P00177442, P00116842, P...
1000003    [P00128042, P00112142, P00182742, P00110742, P...
1000004    [P00184942, P00046742, P00329542, P00114942, P...
1000005    [P00145042, P00324442, P00036842, P00173342, P...
                                 ...                        
1006036    [P00294442, P00118342, P00243942, P00156742, P...
1006037    [P00177442, P00087042, P00025442, P00086442, P...
1006038                    [P00034742, P00086042, P00109542]
1006039    [P00088542, P00254242, P00202742, P00085942, P...
1006040    [P00148642, P00059442, P00024142, P00192042, P...
Name: Product_ID, Length: 5870, dtype: object

# Build model

Associate Rule Mining

In [5]:
# Encode data as transaction matrix
!pip3 install mlxtend --upgrade
from mlxtend.preprocessing import TransactionEncoder

model_trans = TransactionEncoder()
model_trans_arr= model_trans.fit(dfPurchaseList).transform(dfPurchaseList)
dfPurchase = pd.DataFrame(model_trans_arr, columns=model_trans.columns_)
dfPurchase

,P00000142,P00000242,P00000642,P00001042,P00001142,P00001542,P00002142,P00002242,P00003442,P00004242,...,P0096442,P0096542,P0096642,P0096742,P0096842,P0097342,P0099042,P0099742,P0099842,P0099942
0,True,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,False,False,True,False,True,False,False,False,True,False,...,False,True,False,True,False,True,False,False,False,False
5866,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5867,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5868,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
%%time
# Determine the items and itemsets with at least 3% support (generates all itemsets and then filters by support)

from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(dfPurchase, min_support=0.03, use_colnames=True)
frequent_itemsets

CPU times: user 3.46 s, sys: 1.12 s, total: 4.58 s
Wall time: 5.15 s


,support,itemsets
0,0.196252,(P00000142)
1,0.064055,(P00000242)
2,0.087223,(P00000642)
3,0.085690,(P00001042)
4,0.098467,(P00001142)
...,...,...
4236,0.042078,"(P00145042, P00237542, P00270942)"
4237,0.033560,"(P00145042, P00242742, P00270942)"
4238,0.033390,"(P00184942, P00237542, P00270942)"
4239,0.036457,"(P00237542, P00242742, P00270942)"


In [7]:
%%time
# Speed up by using pattern fragment growth method for mining frequent itemsets (uses density to find good itemsets)

from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(dfPurchase, min_support=0.03, use_colnames=True)
frequent_itemsets

CPU times: user 2.23 s, sys: 50.5 ms, total: 2.28 s
Wall time: 2.53 s


,support,itemsets
0,0.275128,(P00025442)
1,0.245315,(P00184942)
2,0.239523,(P00059442)
3,0.218228,(P00110842)
4,0.212266,(P00102642)
...,...,...
4236,0.030664,"(P00165442, P00073842)"
4237,0.030835,"(P00165442, P00237542)"
4238,0.030664,"(P00165442, P00110742)"
4239,0.032198,"(P00165442, P00145042)"


In [14]:
# Evaluate the metrics, and filter the items and itemsets that have at least 65% confidence

from mlxtend.frequent_patterns import association_rules

a_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.55)
a_rules.sort_values(by=['confidence'],ascending=False,inplace=True)
a_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,"(P00102642, P00270942)",(P00057642),0.065928,0.250426,0.039353,0.596899,2.383536,0.022842,1.859520,0.621425
10,"(P00270942, P00329542)",(P00057642),0.057411,0.250426,0.034242,0.596439,2.381699,0.019865,1.857401,0.615466
29,"(P00046742, P00057942)",(P00110742),0.056048,0.274617,0.033049,0.589666,2.147232,0.017658,1.767786,0.566007
23,"(P00073842, P00112542)",(P00110742),0.052300,0.274617,0.030664,0.586319,2.135046,0.016302,1.753486,0.560964
27,"(P00057642, P00057942)",(P00110742),0.057240,0.274617,0.033049,0.577381,2.102498,0.017330,1.716400,0.556213
30,(P00016842),(P00270942),0.053663,0.196252,0.030835,0.574603,2.927882,0.020303,1.889407,0.695794
3,"(P00114942, P00010742)",(P00057642),0.064395,0.250426,0.036797,0.571429,2.281827,0.020671,1.749006,0.600419
9,"(P00057642, P00329542)",(P00270942),0.059966,0.196252,0.034242,0.571023,2.909638,0.022473,1.873637,0.698182
19,"(P00127642, P00270942)",(P00057642),0.057581,0.250426,0.032879,0.571006,2.280139,0.018459,1.747283,0.595733
25,"(P00243942, P00270942)",(P00057642),0.054344,0.250426,0.031005,0.570533,2.278250,0.017396,1.745359,0.593310


In [9]:
# Learn the lowest confidence percentage
a_rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.0)
a_rules.sort_values(by=['confidence'],ascending=True,inplace=True)
a_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
4972,(P00025442),"(P00113242, P00112142)",0.275128,0.058262,0.030153,0.109598,1.881104,0.014124,1.057654,0.646179
6142,(P00025442),"(P00111142, P00086442)",0.275128,0.059284,0.030153,0.109598,1.848671,0.013843,1.056506,0.633313
8351,(P00025442),"(P00046742, P00243942)",0.275128,0.056388,0.030153,0.109598,1.943618,0.014639,1.059758,0.669767
3003,(P00110742),"(P00112542, P00242742)",0.274617,0.057240,0.030153,0.109801,1.918258,0.014434,1.059044,0.659918
552,(P00110742),"(P00000142, P00110842)",0.274617,0.063203,0.030153,0.109801,1.737290,0.012797,1.052346,0.585058


In [10]:
a_rules["confidence"].mean()

0.2887974378553135

# Takeways

- The lowest confidence is 1%, between purchasing P00114942 and P00028442; after purchasing P00025442.
- The highest confidence is 59%, between purchasing P00057642; after purchasing P00329542, P00270942 and P00102642
- The mean of confidence for this dataset is 28%, we can deduce that the most of the relations that we concluded are not strong enough to make a definite suggestion.
- The reason why I chose 3% support threshold is that with a lower threshold, the Python kernel would crash in both local and Colab.